<a href="https://colab.research.google.com/github/HamedGholampour/Music-Recommender-System-With-Pyspark/blob/main/Music_Recommende_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **WELCOME TO THIS NOTEBOOK**

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Let's install pyspark

In [12]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Importing the modules

In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, desc , col, max
from pyspark.ml.feature import  StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder

Creating the spark session


In [14]:
spark = SparkSession.builder.appName("lastfm").getOrCreate()

# Loading the dataset

In [15]:
file_path = '/content/drive/MyDrive/Dataset/listenings.csv'
# file_path = '/content/drive/MyDrive/dataset/dataset/dataset/listenings.csv'
df_listenings = spark.read.format('csv').option('header',True).option('inferSchema',True).load(file_path) #data frame, header will infer column types from csv
df_listenings.show()

+-----------+-------------+--------------------+---------------+--------------------+
|    user_id|         date|               track|         artist|               album|
+-----------+-------------+--------------------+---------------+--------------------+
|000Silenced|1299680100000|           Price Tag|       Jessie J|         Who You Are|
|000Silenced|1299679920000|Price Tag (Acoust...|       Jessie J|           Price Tag|
|000Silenced|1299679440000|Be Mine! (Ballad ...|          Robyn|            Be Mine!|
|000Silenced|1299679200000|            Acapella|          Kelis|            Acapella|
|000Silenced|1299675660000|   I'm Not Invisible|      The Tease|   I'm Not Invisible|
|000Silenced|1297511400000|Bounce (Feat NORE...|       MSTRKRFT|         Fist of God|
|000Silenced|1294498440000|Don't Stop The Mu...|        Rihanna|Addicted 2 Bassli...|
|000Silenced|1292438340000|               ObZen|      Meshuggah|               ObZen|
|000Silenced|1292437740000|   Yama's Messengers|      


# Cleaning tables

In [17]:
df_listenings = df_listenings.drop('date')#drop date and time col from data frame
df_listenings.show() #has information about each user, which track, artist and album they listened to

+-----------+--------------------+---------------+--------------------+
|    user_id|               track|         artist|               album|
+-----------+--------------------+---------------+--------------------+
|000Silenced|           Price Tag|       Jessie J|         Who You Are|
|000Silenced|Price Tag (Acoust...|       Jessie J|           Price Tag|
|000Silenced|Be Mine! (Ballad ...|          Robyn|            Be Mine!|
|000Silenced|            Acapella|          Kelis|            Acapella|
|000Silenced|   I'm Not Invisible|      The Tease|   I'm Not Invisible|
|000Silenced|Bounce (Feat NORE...|       MSTRKRFT|         Fist of God|
|000Silenced|Don't Stop The Mu...|        Rihanna|Addicted 2 Bassli...|
|000Silenced|               ObZen|      Meshuggah|               ObZen|
|000Silenced|   Yama's Messengers|         Gojira|The Way of All Flesh|
|000Silenced|On the Brink of E...|   Napalm Death|Time Waits For No...|
|000Silenced|On the Brink of E...|   Napalm Death|Time Waits For

In [18]:
df_listenings = df_listenings.na.drop() # removes null values in the row
df_listenings.show()

+-----------+--------------------+---------------+--------------------+
|    user_id|               track|         artist|               album|
+-----------+--------------------+---------------+--------------------+
|000Silenced|           Price Tag|       Jessie J|         Who You Are|
|000Silenced|Price Tag (Acoust...|       Jessie J|           Price Tag|
|000Silenced|Be Mine! (Ballad ...|          Robyn|            Be Mine!|
|000Silenced|            Acapella|          Kelis|            Acapella|
|000Silenced|   I'm Not Invisible|      The Tease|   I'm Not Invisible|
|000Silenced|Bounce (Feat NORE...|       MSTRKRFT|         Fist of God|
|000Silenced|Don't Stop The Mu...|        Rihanna|Addicted 2 Bassli...|
|000Silenced|               ObZen|      Meshuggah|               ObZen|
|000Silenced|   Yama's Messengers|         Gojira|The Way of All Flesh|
|000Silenced|On the Brink of E...|   Napalm Death|Time Waits For No...|
|000Silenced|On the Brink of E...|   Napalm Death|Time Waits For

In [19]:
rows = df_listenings.count()
cols = len(df_listenings.columns)
print(rows,cols)

13758905 4



# Let's Perform some aggregation
to see how many times each user has listened to specific track


In [20]:
#in order to make a model, we need to know how many times a user has listened to each song
df_listenings_agg = df_listenings.select('user_id', 'track').groupby('user_id', 'track').agg(count('*').alias('count')).orderBy('user_id')
df_listenings_agg.show()

+-------+--------------------+-----+
|user_id|               track|count|
+-------+--------------------+-----+
| --Seph| White Winter Hymnal|    3|
| --Seph|         The Funeral|    1|
| --Seph|Hope There's Someone|    1|
| --Seph|         The Painter|    1|
| --Seph|          Nightmares|    1|
| --Seph|            War Pigs|    1|
| --Seph|                 F12|    1|
| --Seph|                Team|    1|
| --Seph|          Je te veux|    1|
| --Seph|               Radio|    1|
| --Seph|   All I Want Is You|    1|
| --Seph|    Little by Little|    2|
| --Seph|          Ode to Joy|    1|
| --Seph|In the Hall of th...|    1|
| --Seph|   Hey There Delilah|    1|
| --Seph|   Let's Call It Off|    1|
| --Seph|               Leloo|    1|
| --Seph|             Pack Up|    1|
| --Seph|           Introitus|    1|
| --Seph|        The Leanover|    1|
+-------+--------------------+-----+
only showing top 20 rows



In [22]:
row = df_listenings_agg.count()
col = len(df_listenings_agg.columns)
print(row,col)

9930128 3


In [23]:
#we want to decrement our row size, its almost 10 million, lets reduce it inorder to process our data much faster
df_listenings_agg = df_listenings_agg.limit(20000)


# Let's convert the user id and track columns into unique integers




In [24]:
#We want to use StringIndexer to convert userID and track to unique, integer values
#StringIndexer encodes a string column of labels to a column of label indices
old_strindexer = [StringIndexer(inputCol = col, outputCol = col + '_index').fit(df_listenings_agg) for col in list(set(df_listenings_agg.columns)- set(['count']))]
indexer = [curr_strindexer.setHandleInvalid("keep") for curr_strindexer in old_strindexer]
pipeline = Pipeline(stages = indexer)
#the dataframe will be called data
#Transformers convert one dataframe into another either by updating the current values of a particular column (like converting categorical columns to numeric) or mapping it to some other values by using a defined logic.
#An Estimator implements the fit() method on a dataframe and produces a model.
data = pipeline.fit(df_listenings_agg).transform(df_listenings_agg)
data.show()

+-------+--------------------+-----+-------------+-----------+
|user_id|               track|count|user_id_index|track_index|
+-------+--------------------+-----+-------------+-----------+
| --Seph|          Nightmares|    1|         69.0|    10600.0|
| --Seph|Virus (Luke Fair ...|    1|         69.0|    15893.0|
| --Seph|Airplanes [feat H...|    1|         69.0|      521.0|
| --Seph|Belina (Original ...|    1|         69.0|     3280.0|
| --Seph|              Monday|    1|         69.0|      334.0|
| --Seph|Hungarian Dance No 5|    1|         69.0|     7555.0|
| --Seph|       Life On Mars?|    1|         69.0|     1164.0|
| --Seph|  California Waiting|    1|         69.0|      195.0|
| --Seph|       Phantom Pt II|    1|         69.0|     1378.0|
| --Seph|   Summa for Strings|    1|         69.0|    13737.0|
| --Seph|      Hour for magic|    2|         69.0|     7492.0|
| --Seph|Hungarian Rhapsod...|    1|         69.0|     7556.0|
| --Seph|     The Way We Were|    1|         69.0|    1

In [25]:
data = data.select('user_id_index', 'track_index', 'count').orderBy('user_id_index')



In [26]:
data.show()

+-------------+-----------+-----+
|user_id_index|track_index|count|
+-------------+-----------+-----+
|          0.0|    10628.0|    1|
|          0.0|     3338.0|    1|
|          0.0|    12168.0|    1|
|          0.0|    11626.0|    2|
|          0.0|    10094.0|    4|
|          0.0|      427.0|    1|
|          0.0|    16878.0|    1|
|          0.0|    11722.0|    1|
|          0.0|    15074.0|    1|
|          0.0|     1359.0|    1|
|          0.0|     5874.0|    1|
|          0.0|    11184.0|    1|
|          0.0|     2372.0|    2|
|          0.0|    14316.0|    1|
|          0.0|     5346.0|    1|
|          0.0|    11194.0|    1|
|          0.0|     2241.0|    1|
|          0.0|     2864.0|    1|
|          0.0|     2663.0|    4|
|          0.0|     6064.0|    1|
+-------------+-----------+-----+
only showing top 20 rows



# Train and Test data

In [27]:
(training, test) = data.randomSplit([0.5,0.5])

# Let's Create our Model

In [28]:


USERID = "user_id_index"
TRACK = "track_index"
COUNT = "count"

als = ALS(maxIter = 5, regParam = 0.01, userCol = USERID, itemCol = TRACK, ratingCol = COUNT)
# Alternating Least Squares algorithm
print(als)
model = als.fit(training)

predictions = model.transform(test)

ALS_17a46ff50789




> Indented block


# Generate top 10 Track recommendations for each user

In [29]:
recs = model.recommendForAllUsers(10)

In [30]:
recs.show()

+-------------+--------------------+
|user_id_index|     recommendations|
+-------------+--------------------+
|            0|[{225, 20.841595}...|
|            1|[{11940, 42.26875...|
|            2|[{11940, 29.74676...|
|            3|[{15427, 8.010557...|
|            4|[{308, 5.829543},...|
|            5|[{11940, 52.52713...|
|            6|[{1694, 9.345171}...|
|            7|[{16908, 18.20093...|
|            8|[{11940, 36.43995...|
|            9|[{14825, 7.183723...|
|           10|[{180, 11.222249}...|
|           11|[{1694, 8.721532}...|
|           12|[{308, 7.862448},...|
|           13|[{14825, 12.68514...|
|           14|[{16908, 12.79042...|
|           15|[{180, 10.053168}...|
|           16|[{180, 12.731135}...|
|           17|[{5292, 8.430169}...|
|           18|[{11940, 46.34702...|
|           19|[{225, 18.270203}...|
+-------------+--------------------+
only showing top 20 rows



In [31]:
recs.take(1)

[Row(user_id_index=0, recommendations=[Row(track_index=225, rating=20.841594696044922), Row(track_index=308, rating=19.851253509521484), Row(track_index=251, rating=18.264970779418945), Row(track_index=4461, rating=15.655689239501953), Row(track_index=524, rating=13.046407699584961), Row(track_index=180, rating=11.47872257232666), Row(track_index=16478, rating=10.437127113342285), Row(track_index=4961, rating=10.437127113342285), Row(track_index=4701, rating=10.437127113342285), Row(track_index=873, rating=10.437127113342285)])]